In [1]:
# Initial imports.
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from imblearn.metrics import classification_report_imbalanced
import numpy as np
import pymysql

In [2]:
#conn=pymysql.connect(host='localhost',port=int(3306),user='root',passwd='101Monkeybob!',db='Final_Project')
#df=pd.read_sql_query("SELECT * FROM 'house' ",conn)


In [3]:
# Import Clean House Data
house_data = pd.read_csv("Raw Data/clean_house_data.csv",low_memory = False)
PRIME_data = pd.read_csv('Raw Data/PRIME_data.csv', low_memory = False)
imp_df = pd.DataFrame(house_data)
prime_df = pd.DataFrame(PRIME_data)
imp_df.head()

,BATHRM,HF_BATHRM,AC,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,LONGITUDE,LATITUDE,QUADRANT,PRICE,ZIPCODE,WARD,PRIME
0,3,1,Y,9,5,2016.0,2.0,4,1680,-77.040678,38.914684,NW,2100000.0,20009.0,2,3.750
1,3,1,Y,8,5,2006.0,2.0,3,1680,-77.040629,38.914683,NW,1602000.0,20009.0,2,7.875
2,3,2,Y,10,5,2010.0,1.0,4,2196,-77.039715,38.914331,NW,1950000.0,20009.0,2,3.250
3,3,1,Y,8,4,2011.0,2.0,1,1627,-77.040129,38.915408,NW,1050000.0,20009.0,2,3.250
4,3,1,Y,7,3,2018.0,2.0,1,1424,-77.039903,38.915017,NW,1430000.0,20009.0,2,5.125


In [4]:
imp_df = imp_df.drop(columns=['LONGITUDE','LATITUDE'])
imp_df.dtypes

BATHRM          int64
HF_BATHRM       int64
AC             object
ROOMS           int64
BEDRM           int64
SALEDATE      float64
KITCHENS      float64
FIREPLACES      int64
LANDAREA        int64
QUADRANT       object
PRICE         float64
ZIPCODE       float64
WARD            int64
PRIME         float64
dtype: object

In [5]:
imp_cat = imp_df.dtypes[imp_df.dtypes == "object"].index.tolist()


In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(imp_df[imp_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(imp_cat)
encode_df.head()

C:\Users\FrantheMan\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,AC_0,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [7]:
encode_df = encode_df.drop(columns = 'AC_0')
# Merge one-hot encoded features and drop the originals
imp_df = imp_df.merge(encode_df,left_index=True, right_index=True)
imp_df = imp_df.drop(imp_cat,1)
imp_df.head()

C:\Users\FrantheMan\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,PRICE,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW
0,3,1,9,5,2016.0,2.0,4,1680,2100000.0,20009.0,2,3.750,0.0,1.0,0.0,1.0,0.0,0.0
1,3,1,8,5,2006.0,2.0,3,1680,1602000.0,20009.0,2,7.875,0.0,1.0,0.0,1.0,0.0,0.0
2,3,2,10,5,2010.0,1.0,4,2196,1950000.0,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
3,3,1,8,4,2011.0,2.0,1,1627,1050000.0,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
4,3,1,7,3,2018.0,2.0,1,1424,1430000.0,20009.0,2,5.125,0.0,1.0,0.0,1.0,0.0,0.0


In [8]:
index = imp_df[imp_df.PRICE >= 1500000].index
index2 = imp_df[imp_df.PRICE <= 25000].index
imp_df = imp_df.drop(index)
imp_df = imp_df.drop(index2)


In [9]:
print(imp_df.PRICE.max())
print(imp_df.PRICE.min())


1499500.0
25007.0


In [10]:
#Binning Price data
bins = [25000, 50000, 75000,100000, 125000, 150000, 175000, 200000, 225000,250000,
        275000,300000,325000,350000,375000,400000,425000,450000,475000,500000,
        525000,550000,575000,600000,625000,650000,675000,700000,725000,750000,
        775000, 800000,825000,850000,875000,900000,925000,950000,975000,1000000,
        1025000,1050000,1075000,1100000,1125000,1150000,1175000,1200000,1225000,1250000,
        1275000,1300000,1325000,1350000,1375000,1400000,1425000, 1450000,1475000,1500000]
labels = [x*1 for x in range(1,60)]
cut = pd.cut(imp_df['PRICE'], bins=bins, labels=labels)
cut_df = pd.DataFrame(cut)
cut_df['REAL_PRICE'] = imp_df.PRICE
cut_df = cut_df.rename(columns={'PRICE':'INDEX_BIN'})
cut_df.head(50)

,INDEX_BIN,REAL_PRICE
3,41,1050000.0
4,57,1430000.0
5,52,1325000.0
6,49,1240000.0
7,23,592250.0
8,1,33232.0
9,36,907400.0
10,42,1065000.0
11,43,1100000.0
13,48,1210000.0


In [11]:
imp_df = pd.merge(imp_df, cut_df, left_index=True, right_index=True)
imp_df.head()

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,PRICE,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW,INDEX_BIN,REAL_PRICE
3,3,1,8,4,2011.0,2.0,1,1627,1050000.0,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,41,1050000.0
4,3,1,7,3,2018.0,2.0,1,1424,1430000.0,20009.0,2,5.125000,0.0,1.0,0.0,1.0,0.0,0.0,57,1430000.0
5,3,1,5,3,2011.0,2.0,1,1815,1325000.0,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,52,1325000.0
6,3,1,8,3,2010.0,1.0,0,1424,1240000.0,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,49,1240000.0
7,3,1,9,3,2001.0,2.0,1,1424,592250.0,20009.0,2,6.772727,0.0,1.0,0.0,1.0,0.0,0.0,23,592250.0


In [12]:
# Dropping rows with a saledate less than 2000
new_df = imp_df[imp_df.SALEDATE >= 2000]
new_df.head(50)

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,PRICE,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW,INDEX_BIN,REAL_PRICE
3,3,1,8,4,2011.0,2.0,1,1627,1050000.0,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,41,1050000.0
4,3,1,7,3,2018.0,2.0,1,1424,1430000.0,20009.0,2,5.125000,0.0,1.0,0.0,1.0,0.0,0.0,57,1430000.0
5,3,1,5,3,2011.0,2.0,1,1815,1325000.0,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,52,1325000.0
6,3,1,8,3,2010.0,1.0,0,1424,1240000.0,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,49,1240000.0
7,3,1,9,3,2001.0,2.0,1,1424,592250.0,20009.0,2,6.772727,0.0,1.0,0.0,1.0,0.0,0.0,23,592250.0
8,3,1,14,5,2016.0,1.0,3,2090,33232.0,20009.0,2,3.750000,0.0,1.0,0.0,1.0,0.0,0.0,1,33232.0
9,1,0,6,3,2006.0,1.0,0,2090,907400.0,20009.0,2,7.875000,0.0,1.0,0.0,1.0,0.0,0.0,36,907400.0
10,2,1,5,3,2012.0,1.0,1,2090,1065000.0,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,42,1065000.0
11,2,1,8,3,2012.0,1.0,2,2090,1100000.0,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,43,1100000.0
13,3,1,11,3,2013.0,2.0,3,1853,1210000.0,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,48,1210000.0


In [13]:
remove_n = 35000
drop_indices = np.random.choice(new_df.index, remove_n, replace=False)
new_df = new_df.drop(drop_indices)
new_df

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,PRICE,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW,INDEX_BIN,REAL_PRICE
8,3,1,14,5,2016.0,1.0,3,2090,33232.0,20009.0,2,3.750000,0.0,1.0,0.0,1.0,0.0,0.0,1,33232.0
9,1,0,6,3,2006.0,1.0,0,2090,907400.0,20009.0,2,7.875000,0.0,1.0,0.0,1.0,0.0,0.0,36,907400.0
22,3,1,9,4,2011.0,2.0,2,1971,846000.0,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,33,846000.0
38,2,1,11,4,2013.0,2.0,2,1183,1155000.0,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,46,1155000.0
41,3,1,9,3,2005.0,2.0,0,1795,900000.0,20009.0,2,6.375000,0.0,1.0,0.0,1.0,0.0,0.0,35,900000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55000,2,1,6,3,2017.0,1.0,0,1834,350000.0,20032.0,8,4.250000,0.0,1.0,0.0,0.0,0.0,1.0,13,350000.0
55018,1,1,9,3,2001.0,1.0,0,2799,120000.0,20032.0,8,6.772727,0.0,1.0,0.0,0.0,1.0,0.0,4,120000.0
55021,1,1,7,3,2012.0,1.0,0,1919,215000.0,20032.0,8,3.250000,0.0,1.0,0.0,0.0,1.0,0.0,8,215000.0
55022,2,1,7,3,2005.0,1.0,0,2513,205000.0,20032.0,8,6.375000,0.0,1.0,0.0,0.0,1.0,0.0,8,205000.0


In [14]:
new_df = new_df.drop(columns=['REAL_PRICE','PRICE'])
new_df

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW,INDEX_BIN
8,3,1,14,5,2016.0,1.0,3,2090,20009.0,2,3.750000,0.0,1.0,0.0,1.0,0.0,0.0,1
9,1,0,6,3,2006.0,1.0,0,2090,20009.0,2,7.875000,0.0,1.0,0.0,1.0,0.0,0.0,36
22,3,1,9,4,2011.0,2.0,2,1971,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,33
38,2,1,11,4,2013.0,2.0,2,1183,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,46
41,3,1,9,3,2005.0,2.0,0,1795,20009.0,2,6.375000,0.0,1.0,0.0,1.0,0.0,0.0,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55000,2,1,6,3,2017.0,1.0,0,1834,20032.0,8,4.250000,0.0,1.0,0.0,0.0,0.0,1.0,13
55018,1,1,9,3,2001.0,1.0,0,2799,20032.0,8,6.772727,0.0,1.0,0.0,0.0,1.0,0.0,4
55021,1,1,7,3,2012.0,1.0,0,1919,20032.0,8,3.250000,0.0,1.0,0.0,0.0,1.0,0.0,8
55022,2,1,7,3,2005.0,1.0,0,2513,20032.0,8,6.375000,0.0,1.0,0.0,0.0,1.0,0.0,8


In [15]:
# Define the features 
X = new_df.copy()
X = X.drop("INDEX_BIN", axis = 1)
X.head()

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW
8,3,1,14,5,2016.0,1.0,3,2090,20009.0,2,3.750,0.0,1.0,0.0,1.0,0.0,0.0
9,1,0,6,3,2006.0,1.0,0,2090,20009.0,2,7.875,0.0,1.0,0.0,1.0,0.0,0.0
22,3,1,9,4,2011.0,2.0,2,1971,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
38,2,1,11,4,2013.0,2.0,2,1183,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
41,3,1,9,3,2005.0,2.0,0,1795,20009.0,2,6.375,0.0,1.0,0.0,1.0,0.0,0.0


In [16]:
# Define target set
y = new_df['INDEX_BIN'].ravel()
y[:5]

[1, 36, 33, 46, 35]
Categories (59, int64): [1 < 2 < 3 < 4 ... 56 < 57 < 58 < 59]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [18]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators = 100, random_state = 1)

In [20]:
#Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [21]:
y_pred = rf_model.predict(X_test_scaled)

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          1       0.12      0.10      1.00      0.11      0.32      0.09        10
          2       0.22      0.19      0.99      0.20      0.44      0.18        26
          3       0.17      0.23      0.98      0.19      0.47      0.21        44
          4       0.23      0.20      0.98      0.22      0.44      0.18        70
          5       0.22      0.21      0.97      0.21      0.45      0.19       111
          6       0.13      0.14      0.97      0.14      0.37      0.12        93
          7       0.11      0.12      0.97      0.12      0.34      0.11        92
          8       0.11      0.15      0.96      0.13      0.38      0.13        82
          9       0.17      0.18      0.96      0.17      0.42      0.16       104
         10       0.17      0.17      0.97      0.17      0.41      0.15        83
         11       0.21      0.19      0.97      0.20      0.42      0.17       113
   

In [23]:
acc_score = accuracy_score(y_test, y_pred)
acc_score

0.10739942528735633

In [24]:
# Displaying results
print("Confusion Matrix")
#display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))

Confusion Matrix
Accuracy Score : 0.10739942528735633
Classification Report
              precision    recall  f1-score   support

           1       0.12      0.10      0.11        10
           2       0.22      0.19      0.20        26
           3       0.17      0.23      0.19        44
           4       0.23      0.20      0.22        70
           5       0.22      0.21      0.21       111
           6       0.13      0.14      0.14        93
           7       0.11      0.12      0.12        92
           8       0.11      0.15      0.13        82
           9       0.17      0.18      0.17       104
          10       0.17      0.17      0.17        83
          11       0.21      0.19      0.20       113
          12       0.24      0.22      0.23        90
          13       0.14      0.13      0.13       102
          14       0.03      0.04      0.04        73
          15       0.09      0.07      0.08        95
          16       0.10      0.12      0.11        66
     

In [25]:
# Calc feature importancy 
importancies = rf_model.feature_importances_
importancies

array([0.05444404, 0.05706721, 0.11056789, 0.07005058, 0.13319161,
       0.02376172, 0.04351131, 0.25024203, 0.07218249, 0.04598416,
       0.08349381, 0.01330099, 0.01352565, 0.01097009, 0.00671201,
       0.00905353, 0.00194087])

In [26]:
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.2502420263061911, 'LANDAREA'),
 (0.1331916060118556, 'SALEDATE'),
 (0.1105678940592887, 'ROOMS'),
 (0.08349381039729094, 'PRIME'),
 (0.07218248779074234, 'ZIPCODE'),
 (0.07005058462218636, 'BEDRM'),
 (0.057067210987032393, 'HF_BATHRM'),
 (0.05444404494486076, 'BATHRM'),
 (0.04598416155626075, 'WARD'),
 (0.04351131315739465, 'FIREPLACES'),
 (0.023761716705689002, 'KITCHENS'),
 (0.013525652830407422, 'AC_Y'),
 (0.013300989709733141, 'AC_N'),
 (0.010970085994998782, 'QUADRANT_NE'),
 (0.009053532318489354, 'QUADRANT_SE'),
 (0.0067120085493718225, 'QUADRANT_NW'),
 (0.0019408740582067912, 'QUADRANT_SW')]